## Grid Objectives
Iterating between min and max for each column

### Glossary
- **task**: Refers to the set of values (row) and corresponding keys to be aimed at sequentially.
- **objective**: Refers to one key (column) and respective value to be aimed at simultaneously during a task.
- **targets**: Refers to one file containing a multiple of objectives and tasks for a fixed number of each, respectively. 

In [1]:
import itertools
import json
import numpy as np
import os
import pandas as pd

In [2]:
#Features between 0 and 1: 
"""
normalized_feature_names = ['ratio_variants_per_number_of_traces', 'trace_len_hist1', 'trace_len_hist2',
                            'trace_len_hist3', 'trace_len_hist4', 'trace_len_hist5', 'trace_len_hist7',
                            'trace_len_hist8', 'trace_len_hist9', 'ratio_most_common_variant', 
                            'ratio_top_1_variants', 'ratio_top_5_variants', 'ratio_top_10_variants', 
                            'ratio_top_20_variants', 'ratio_top_50_variants', 'ratio_top_75_variants', 
                            'epa_normalized_variant_entropy', 'epa_normalized_sequence_entropy', 
                            'epa_normalized_sequence_entropy_linear_forgetting', 'epa_normalized_sequence_entropy_exponential_forgetting']
"""
normalized_feature_names = ['ratio_variants_per_number_of_traces', 'ratio_most_common_variant', 
                            'ratio_top_10_variants', 'epa_normalized_variant_entropy', 'epa_normalized_sequence_entropy', 
                            'epa_normalized_sequence_entropy_linear_forgetting', 'epa_normalized_sequence_entropy_exponential_forgetting']
def abbrev_obj_keys(obj_keys):
    abbreviated_keys = []
    for obj_key in obj_keys:
        key_slices = obj_key.split("_")
        chars = []
        for key_slice in key_slices:
            for idx, single_char in enumerate(key_slice):
                if idx == 0 or single_char.isdigit():
                    chars.append(single_char)
        abbreviated_key = ''.join(chars)
        abbreviated_keys.append(abbreviated_key)
    return '_'.join(abbreviated_keys) 

In [16]:
def write_generator_targets(targets_path, objectives=["ratio_top_20_variants", "epa_normalized_sequence_entropy_linear_forgetting"]):
    first_dir = os.path.split(targets_path[3:])[-1].replace(".csv","")
    second_dir = first_dir.replace("grid_","").replace("objectives","")

    targets = [
      {
        'pipeline_step': 'event_logs_generation',
        "targets": {"input_path": targets_path[3:],
            "objectives": objectives},
        'config_space': {
            'mode': [5, 20],
            'sequence': [0.01, 1],
            'choice': [0.01, 1],
            'parallel': [0.01, 1],
            'loop': [0.01, 1],
            'silent': [0.01, 1],
            'lt_dependency': [0.01, 1],
            'num_traces': [10, 10001],
            'duplicate': [0],
            'or': [0]
          },
          'system_params':{
              'output_path':'output/generated/grid_2obj',
              'n_trials': 200
          }
      },
      {
        'pipeline_step': 'feature_extraction',
        'input_path': os.path.join('output','features', 'generated', 'grid_2obj', first_dir, second_dir),
        "feature_params": {"feature_set":["ratio_variants_per_number_of_traces","ratio_most_common_variant","ratio_top_10_variants","epa_normalized_variant_entropy","epa_normalized_sequence_entropy","epa_normalized_sequence_entropy_linear_forgetting","epa_normalized_sequence_entropy_exponential_forgetting"]},
        'output_path': 'output/plots',
        'real_eventlog_path': 'data/BaselineED_feat.csv',
        'plot_type': 'boxplot'
      },
      {
        "pipeline_step": "benchmark_test",
        "benchmark_test": "discovery",
        "input_path": os.path.join('output', 'generated', 'grid_2obj', first_dir, second_dir),
        "output_path":"output",
        "miners" : ["heu", "imf", "ilp"]
      }
    ]

    #print("targets:", targets[1]['input_path'])
    output_path = os.path.join('..', 'config_files','algorithm',f'grid_{len(objectives)}obj')
    os.makedirs(output_path, exist_ok=True)
    output_path = os.path.join(output_path, f'generator_{os.path.split(targets_path)[-1].split(".")[0]}.json') 
    with open(output_path, 'w') as f:
        json.dump(targets, f, ensure_ascii=False)
    print(f"Saved targets config in {output_path}")
    
    return targets

def create_objectives_grid(objectives, n_para_obj=2):
    parameters_o = "objectives, "
    if n_para_obj==len(objectives):
        targets = [tuple(sorted(objectives))]
        print(len(targets), targets)
        parameters = get_ranges_from_data(sorted(objectives))
        tasks = eval(f"list(itertools.product({parameters}))")
        #tasks = eval(f"list(itertools.product({(parameters*n_para_obj)[:-2]}))")
    else: 
        if n_para_obj==1:
            targets = [[exp] for exp in objectives]
        else:
            targets = eval(f"[exp for exp in list(itertools.product({(parameters_o*n_para_obj)[:-2]})) if exp[0]!=exp[1]]")
        targets = list(set([tuple(sorted(exp)) for exp in targets]))
        parameters = "np.around(np.arange(0.0, 1.5,0.5),2), "
        tasks = eval(f"list(itertools.product({(parameters*n_para_obj)[:-2]}))")
    print("TASKS", type(parameters), type(n_para_obj), parameters*n_para_obj)
    print(len(targets), targets)

    tasks = [(f'task_{i+1}',)+task for i, task in enumerate(tasks)]
    print(len(tasks))
    for exp in targets:
        df = pd.DataFrame(data=tasks, columns=["task", *exp])
        targets_path = os.path.join('..','data', f'grid_{n_para_obj}obj')
        os.makedirs(targets_path, exist_ok=True)
        targets_path = os.path.join(targets_path, f"grid_{len(df.columns)-1}objectives_{abbrev_obj_keys(exp)}.csv") 
        df.to_csv(targets_path, index=False)
        print(f"Saved targets in {targets_path}")
        write_generator_targets(targets_path, objectives=exp)
    #df.to_csv(f"../data/grid_{}objectives_{abbrev_obj_keys(objectives.tolist())}.csv" ,index=False)

exp_test = create_objectives_grid(normalized_feature_names, n_para_obj=2)        
print(exp_test)

TASKS <class 'str'> <class 'int'> np.around(np.arange(0.0, 1.5,0.5),2), np.around(np.arange(0.0, 1.5,0.5),2), 
21 [('mean_variant_occurrence', 'trace_len_coefficient_variation'), ('activities_std', 'eventropy_trace'), ('epa_normalized_variant_entropy', 'ratio_variants_per_number_of_traces'), ('activities_std', 'epa_normalized_variant_entropy'), ('eventropy_trace', 'trace_len_coefficient_variation'), ('ratio_variants_per_number_of_traces', 'trace_len_coefficient_variation'), ('activities_std', 'trace_len_coefficient_variation'), ('eventropy_trace', 'mean_variant_occurrence'), ('activities_std', 'mean_variant_occurrence'), ('epa_normalized_variant_entropy', 'eventropy_trace'), ('mean_variant_occurrence', 'start_activities_median'), ('ratio_variants_per_number_of_traces', 'start_activities_median'), ('eventropy_trace', 'start_activities_median'), ('activities_std', 'start_activities_median'), ('epa_normalized_variant_entropy', 'trace_len_coefficient_variation'), ('epa_normalized_variant_e

## Grid Objectives
Based on real ED ranges.

In [17]:
DF_PATH = "../../shampu/data/bench_baseline_feat.csv"
def get_ranges_from_data(objectives, df_path = DF_PATH):
    #print(objectives)
    dmf = pd.read_csv(DF_PATH, index_col=None)
    dmf = dmf[objectives].describe()
    dmf = dmf.transpose()[['min', 'mean','max']]
    dmf['range'] = dmf.apply(lambda x: tuple([x['min'], x['mean'], x['max']]), axis=1)
    print(dmf['range'])
    #tasks = eval(f"list(itertools.product({(parameters*n_para_obj)[:-2]}))")
    result = [f"np.around({x}, 2)" for x in dmf['range']]
    result = ", ".join(result)
    return result

print(normalized_feature_names)
get_ranges_from_data(normalized_feature_names)

['ratio_variants_per_number_of_traces', 'trace_len_coefficient_variation', 'mean_variant_occurrence', 'activities_std', 'start_activities_median', 'eventropy_trace', 'epa_normalized_variant_entropy']
ratio_variants_per_number_of_traces    (4.081521591249218e-05, 0.4659094439111451, 0....
trace_len_coefficient_variation             (0.0, 0.6838390025070027, 4.744080106525514)
mean_variant_occurrence                (1.001552795031056, 838.6048767068644, 24500.6...
activities_std                             (0.0, 12982.056069959535, 120522.24741658216)
start_activities_median                               (1.0, 7975.705882352941, 150370.0)
eventropy_trace                                        (0.0, 6.2416470588235295, 13.362)
epa_normalized_variant_entropy              (0.0, 0.6773545645863115, 0.899497456838069)
Name: range, dtype: object


'np.around((4.081521591249218e-05, 0.4659094439111451, 0.9984496124031008), 2), np.around((0.0, 0.6838390025070027, 4.744080106525514), 2), np.around((1.001552795031056, 838.6048767068644, 24500.666666666668), 2), np.around((0.0, 12982.056069959535, 120522.24741658216), 2), np.around((1.0, 7975.705882352941, 150370.0), 2), np.around((0.0, 6.2416470588235295, 13.362), 2), np.around((0.0, 0.6773545645863115, 0.899497456838069), 2)'

In [18]:
normalized_feature_names = ['ratio_variants_per_number_of_traces', 'trace_len_coefficient_variation', 'mean_variant_occurrence', 'activities_std', 'start_activities_median', 'eventropy_trace', 'epa_normalized_variant_entropy']
exp_test = create_objectives_grid(normalized_feature_names, n_para_obj=len(normalized_feature_names))        
print(exp_test)

1 [('activities_std', 'epa_normalized_variant_entropy', 'eventropy_trace', 'mean_variant_occurrence', 'ratio_variants_per_number_of_traces', 'start_activities_median', 'trace_len_coefficient_variation')]
activities_std                             (0.0, 12982.056069959535, 120522.24741658216)
epa_normalized_variant_entropy              (0.0, 0.6773545645863115, 0.899497456838069)
eventropy_trace                                        (0.0, 6.2416470588235295, 13.362)
mean_variant_occurrence                (1.001552795031056, 838.6048767068644, 24500.6...
ratio_variants_per_number_of_traces    (4.081521591249218e-05, 0.4659094439111451, 0....
start_activities_median                               (1.0, 7975.705882352941, 150370.0)
trace_len_coefficient_variation             (0.0, 0.6838390025070027, 4.744080106525514)
Name: range, dtype: object
TASKS <class 'str'> <class 'int'> np.around((0.0, 12982.056069959535, 120522.24741658216), 2), np.around((0.0, 0.6773545645863115, 0.8994974568380

### Helper prototypes

In [6]:
df = pd.DataFrame(columns=["log","ratio_top_20_variants", "epa_normalized_sequence_entropy_linear_forgetting"])    

In [7]:
k=0
for i in np.arange(0, 1.1,0.5):
    for j in np.arange(0,0.55,0.5):
        k+=1
        new_entry = pd.Series({'log':f"objective_{k}", "ratio_top_20_variants":round(i,1),
                   "epa_normalized_sequence_entropy_linear_forgetting":round(j,1)})
        df = pd.concat([
                df, 
                pd.DataFrame([new_entry], columns=new_entry.index)]
           ).reset_index(drop=True)
    

/var/folders/d0/btmbyskx4t106_l2zghzln2w0000gn/T/ipykernel_12596/3751377549.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([


In [8]:
df.to_csv("../data/grid_objectives.csv" ,index=False)

## Objectives from real logs
(Feature selection)

In [9]:
bpic_features = pd.read_csv("../data/BaselineED_feat.csv", index_col=None)
#bpic_features = pd.read_csv("../gedi/output/features/real_event_logs.csv", index_col=None)

#bpic_features = bpic_features.drop(['Unnamed: 0'], axis=1)
print(bpic_features.shape)
print(len(bpic_features), " Event-Logs: ", bpic_features.sort_values('log')['log'].unique())

#bpic_features.rename(columns={"variant_entropy":"epa_variant_entropy", "normalized_variant_entropy":"epa_normalized_variant_entropy", "sequence_entropy":"epa_sequence_entropy", "normalized_sequence_entropy":"epa_normalized_sequence_entropy", "sequence_entropy_linear_forgetting":"epa_sequence_entropy_linear_forgetting", "normalized_sequence_entropy_linear_forgetting":"epa_normalized_sequence_entropy_linear_forgetting", "sequence_entropy_exponential_forgetting":"epa_sequence_entropy_exponential_forgetting", "normalized_sequence_entropy_exponential_forgetting":"epa_normalized_sequence_entropy_exponential_forgetting"},
#          errors="raise", inplace=True)

bpic_features.head()
#bpic_features.to_csv("../data/BaselineED_feat.csv", index=False)

(26, 8)
26  Event-Logs:  ['BPIC12' 'BPIC13cp' 'BPIC13inc' 'BPIC13op' 'BPIC14dc_p' 'BPIC14di_p'
 'BPIC14dia_p' 'BPIC15f1' 'BPIC15f2' 'BPIC15f3' 'BPIC15f4' 'BPIC15f5'
 'BPIC16c_p' 'BPIC16wm_p' 'BPIC17' 'BPIC17ol' 'BPIC19' 'BPIC20a' 'BPIC20b'
 'BPIC20c' 'BPIC20d' 'BPIC20e' 'HD' 'RTFMP' 'RWABOCSL' 'SEPSIS']


,log,ratio_variants_per_number_of_traces,ratio_most_common_variant,ratio_top_10_variants,epa_normalized_variant_entropy,epa_normalized_sequence_entropy,epa_normalized_sequence_entropy_linear_forgetting,epa_normalized_sequence_entropy_exponential_forgetting
0,BPIC16wm_p,0.002882,0.295803,0.714106,0.000000,0.000000,0.000000,0.000000
1,BPIC15f5,0.997405,0.001730,0.102076,0.648702,0.603260,0.342410,0.404580
2,BPIC15f1,0.975813,0.006672,0.121768,0.652855,0.610294,0.270241,0.363928
3,BPIC19,0.047562,0.199758,0.946368,0.645530,0.328029,0.320185,0.320282
4,BPIC14dia_p,0.496847,0.037455,0.552836,0.774743,0.608350,0.305614,0.377416


In [10]:
bpic_stats = bpic_features.describe().transpose()
normalized_feature_names = bpic_stats[(bpic_stats['min']>=0)&(bpic_stats['max']<=1)].index.to_list() 
normalized_feature_names = ['ratio_variants_per_number_of_traces', 'ratio_most_common_variant', 
                            'ratio_top_10_variants', 'epa_normalized_variant_entropy', 'epa_normalized_sequence_entropy', 
                            'epa_normalized_sequence_entropy_linear_forgetting', 'epa_normalized_sequence_entropy_exponential_forgetting']
print(normalized_feature_names)
bpic_features[['log']+normalized_feature_names]

['ratio_variants_per_number_of_traces', 'ratio_most_common_variant', 'ratio_top_10_variants', 'epa_normalized_variant_entropy', 'epa_normalized_sequence_entropy', 'epa_normalized_sequence_entropy_linear_forgetting', 'epa_normalized_sequence_entropy_exponential_forgetting']


,log,ratio_variants_per_number_of_traces,ratio_most_common_variant,ratio_top_10_variants,epa_normalized_variant_entropy,epa_normalized_sequence_entropy,epa_normalized_sequence_entropy_linear_forgetting,epa_normalized_sequence_entropy_exponential_forgetting
0,BPIC16wm_p,0.002882,0.295803,0.714106,0.000000,0.000000,0.000000,0.000000
1,BPIC15f5,0.997405,0.001730,0.102076,0.648702,0.603260,0.342410,0.404580
2,BPIC15f1,0.975813,0.006672,0.121768,0.652855,0.610294,0.270241,0.363928
3,BPIC19,0.047562,0.199758,0.946368,0.645530,0.328029,0.320185,0.320282
4,BPIC14dia_p,0.496847,0.037455,0.552836,0.774743,0.608350,0.305614,0.377416
5,BPIC15f2,0.995192,0.002404,0.103365,0.627973,0.602371,0.317217,0.390473
6,BPIC15f3,0.957417,0.010646,0.137686,0.661781,0.605676,0.341521,0.404934
7,BPIC13cp,0.123067,0.331540,0.840619,0.705383,0.310940,0.286515,0.288383
8,BPIC14dc_p,0.048444,0.074944,0.765056,0.470758,0.419266,0.312599,0.326719
9,BPIC20a,0.009429,0.439810,0.950095,0.696474,0.164758,0.085439,0.104389


In [11]:
#Features between 0 and 1: 
def write_generator_bpic_targets(objectives, n_para_obj=2):
    parameters_o = "objectives, "
    targets = eval(f"[exp for exp in list(itertools.product({(parameters_o*n_para_obj)[:-2]})) if exp[0]!=exp[1]]")
    targets = list(set([tuple(sorted(exp)) for exp in targets]))
    for exp in targets:
        targets_path = os.path.join('..','data', 'BaselineED_feat')
        os.makedirs(targets_path, exist_ok=True)
        targets_path = os.path.join(targets_path, f"{len(exp)}_{abbrev_obj_keys(exp)}.csv") 


        first_dir = os.path.split(targets_path[3:])[-1].replace(".csv","")
        second_dir = first_dir.replace("grid_","").replace("objectives","")

        targets = [
          {
            'pipeline_step': 'event_logs_generation',
            "targets": {"input_path": "data/BaselineED_feat.csv",
                "objectives": exp},
            'config_space': {
                'mode': [5, 20],
                'sequence': [0.01, 1],
                'choice': [0.01, 1],
                'parallel': [0.01, 1],
                'loop': [0.01, 1],
                'silent': [0.01, 1],
                'lt_dependency': [0.01, 1],
                'num_traces': [10, 10001],
                'duplicate': [0],
                'or': [0]
              },
            'system_params':{
                'output_path':'output/generated',
                'n_trials': 200
            }
          },
          {
            'pipeline_step': 'feature_extraction',
            'input_path': os.path.join('output', 'features', 'generated', 'BaselineED_feat', first_dir),
            'input_path': os.path.join('output', 'generated', 'BaselineED_feat', first_dir),
            'feature_params': {'feature_set':['simple_stats', 'trace_length', 'trace_variant', 'activities', 'start_activities', 'end_activities', 'eventropies', 'epa_based']},
            'feature_params': {"feature_set":["ratio_variants_per_number_of_traces","ratio_most_common_variant","ratio_top_10_variants","epa_normalized_variant_entropy","epa_normalized_sequence_entropy","epa_normalized_sequence_entropy_linear_forgetting","epa_normalized_sequence_entropy_exponential_forgetting"]},
            'output_path': 'output/plots',
            'real_eventlog_path': 'data/BaselineED_feat.csv',
            'plot_type': 'boxplot'
          },
          {
            "pipeline_step": "benchmark_test",
            "benchmark_test": "discovery",
            "input_path": os.path.join('output', 'generated', 'BaselineED_feat', first_dir),
            "output_path":"output",
            "miners" : ["heu", "imf", "ilp"]
          }
        ]

        output_path = os.path.join('..', 'config_files','algorithm','BaselineED_feat')
        os.makedirs(output_path, exist_ok=True)
        output_path = os.path.join(output_path, f'generator_{os.path.split(targets_path)[-1].split(".")[0]}.json') 

        with open(output_path, 'w') as f:
            json.dump(targets, f, ensure_ascii=False)
        print(f"Saved targets config in {output_path}")
        return targets


def create_objectives_grid(objectives, n_para_obj=2):
    parameters_o = "objectives, "
    targets = eval(f"[exp for exp in list(itertools.product({(parameters_o*n_para_obj)[:-2]})) if exp[0]!=exp[1]]")
    targets = list(set([tuple(sorted(exp)) for exp in targets]))
    print(len(targets))
    
    for exp in targets:
        write_generator_bpic_targets(objectives=exp)
        
exp_test = create_objectives_grid(normalized_feature_names, n_para_obj=2)        
print(exp_test)

21
Saved targets config in ../config_files/algorithm/BaselineED_feat/generator_2_enself_rvpnot.json
Saved targets config in ../config_files/algorithm/BaselineED_feat/generator_2_rmcv_rvpnot.json
Saved targets config in ../config_files/algorithm/BaselineED_feat/generator_2_ense_enself.json
Saved targets config in ../config_files/algorithm/BaselineED_feat/generator_2_ense_enseef.json
Saved targets config in ../config_files/algorithm/BaselineED_feat/generator_2_enve_rvpnot.json
Saved targets config in ../config_files/algorithm/BaselineED_feat/generator_2_enseef_rt10v.json
Saved targets config in ../config_files/algorithm/BaselineED_feat/generator_2_enself_rt10v.json
Saved targets config in ../config_files/algorithm/BaselineED_feat/generator_2_enseef_enve.json
Saved targets config in ../config_files/algorithm/BaselineED_feat/generator_2_rmcv_rt10v.json
Saved targets config in ../config_files/algorithm/BaselineED_feat/generator_2_enself_enve.json
Saved targets config in ../config_files/algo

## Single objective from real logs
(Feature selection)

In [12]:
def write_single_objective_targets(targets_path, objectives=["ratio_top_20_variants", "epa_normalized_sequence_entropy_linear_forgetting"]):
    first_dir = os.path.split(targets_path[3:])[-1].replace(".csv","")
    second_dir = first_dir.replace("grid_","").replace("objectives","")

    targets = [
      {
        'pipeline_step': 'event_logs_generation',
        "targets": {"input_path": targets_path[3:],
            "objectives": objectives},
        'config_space': {
            'mode': [5, 20],
            'sequence': [0.01, 1],
            'choice': [0.01, 1],
            'parallel': [0.01, 1],
            'loop': [0.01, 1],
            'silent': [0.01, 1],
            'lt_dependency': [0.01, 1],
            'num_traces': [10, 10001],
            'duplicate': [0],
            'or': [0]
          },
          'system_params':{
            'output_path':os.path.join('output','generated', 'grid_1obj'),
            'n_trials': 200
        }
      },
      {
        'pipeline_step': 'feature_extraction',
        'input_path': os.path.join('output','features', 'generated', 'grid_1obj', first_dir, second_dir),
        'feature_params': {'feature_set':['simple_stats', 'trace_length', 'trace_variant', 'activities', 'start_activities', 'end_activities', 'eventropies', 'epa_based']},
        'feature_params': {"feature_set":["ratio_variants_per_number_of_traces","ratio_most_common_variant","ratio_top_10_variants","epa_normalized_variant_entropy","epa_normalized_sequence_entropy","epa_normalized_sequence_entropy_linear_forgetting","epa_normalized_sequence_entropy_exponential_forgetting"]},
        'output_path': 'output/plots',
        'real_eventlog_path': 'data/BaselineED_feat.csv',
        'plot_type': 'boxplot'
      },
      {
        "pipeline_step": "benchmark_test",
        "benchmark_test": "discovery",
        "input_path": os.path.join('output', 'generated', 'grid_1obj', first_dir, second_dir),
        "output_path":"output",
        "miners" : ["heu", "imf", "ilp"]
      }
    ]

    #print("targets:", targets)
    output_path = os.path.join('..', 'config_files','algorithm','grid_targets')
    os.makedirs(output_path, exist_ok=True)
    output_path = os.path.join(output_path, f'generator_{os.path.split(targets_path)[-1].split(".")[0]}.json') 
    with open(output_path, 'w') as f:
        json.dump(targets, f, ensure_ascii=False)
    print(f"Saved targets config in {output_path}")
    
    return targets

def create_objectives_grid(objectives, n_para_obj=2):
    parameters_o = "objectives, "
    if n_para_obj==1:
        targets = [[exp] for exp in objectives]
    else:
        targets = eval(f"[exp for exp in list(itertools.product({(parameters_o*n_para_obj)[:-2]})) if exp[0]!=exp[1]]")
    targets = list(set([tuple(sorted(exp)) for exp in targets]))
    print(len(targets), "targets: ", targets)
    
    parameters = "np.around(np.arange(0, 1.1,0.1),2), "
    tasks = eval(f"list(itertools.product({(parameters*n_para_obj)[:-2]}))")
    tasks = [(f'task_{i+1}',)+task for i, task in enumerate(tasks)]
    print(len(tasks))
    for exp in targets:
        df = pd.DataFrame(data=tasks, columns=["task", *exp])
        targets_path = os.path.join('..','data', 'grid_targets')
        os.makedirs(targets_path, exist_ok=True)
        targets_path = os.path.join(targets_path, f"grid_{len(df.columns)-1}objectives_{abbrev_obj_keys(exp)}.csv") 
        df.to_csv(targets_path, index=False)
        print(f"Saved targets in {targets_path}")
        write_single_objective_targets(targets_path, objectives=exp)
    #df.to_csv(f"../data/grid_{}objectives_{abbrev_obj_keys(objectives.tolist())}.csv" ,index=False)
        
exp_test = create_objectives_grid(normalized_feature_names, n_para_obj=1)        
print(exp_test)

7 targets:  [('epa_normalized_sequence_entropy_linear_forgetting',), ('ratio_most_common_variant',), ('epa_normalized_sequence_entropy_exponential_forgetting',), ('epa_normalized_sequence_entropy',), ('ratio_top_10_variants',), ('ratio_variants_per_number_of_traces',), ('epa_normalized_variant_entropy',)]
11
Saved targets in ../data/grid_targets/grid_1objectives_enself.csv
Saved targets config in ../config_files/algorithm/grid_targets/generator_grid_1objectives_enself.json
Saved targets in ../data/grid_targets/grid_1objectives_rmcv.csv
Saved targets config in ../config_files/algorithm/grid_targets/generator_grid_1objectives_rmcv.json
Saved targets in ../data/grid_targets/grid_1objectives_enseef.csv
Saved targets config in ../config_files/algorithm/grid_targets/generator_grid_1objectives_enseef.json
Saved targets in ../data/grid_targets/grid_1objectives_ense.csv
Saved targets config in ../config_files/algorithm/grid_targets/generator_grid_1objectives_ense.json
Saved targets in ../data/g